In [36]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
# To load the environment variable defined in the .env file
from dotenv import load_dotenv
load_dotenv();

In [38]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.calibration import CalibrationDisplay
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc

import src.data.NHLDataManager as DataManager
import src.features.build_features as FeaturesManager

In [40]:
seasons_year = [2015, 2016, 2017, 2018]
season_type = "Regular"
features_data_df = FeaturesManager.build_features(seasons_year, season_type, with_player_stats=True, with_strength_stats=True)
# features_data_df.drop(columns=['Team', 'Shooter', 'Goalie', 'Shooter ID', 'Goalie ID', 'X', 'Y', 'Last event X', 'Last event Y'], inplace=True)


In [41]:
# Q1
features_data_df[['Type', 'Period', 'Period seconds', 'Game seconds', 'st_X', 'st_Y', 'Shot distance', 'Shot angle', 'Shot Type']].sample(10)


,Type,Period,Period seconds,Game seconds,st_X,st_Y,Shot distance,Shot angle,Shot Type
211852,Shot,1,99.0,99,35.0,-17.0,56.612719,-17.474774,Slap Shot
155362,Shot,1,638.0,638,35.0,-19.0,57.245087,-19.384516,Slap Shot
194825,Shot,2,790.0,1990,48.0,-18.0,44.777226,-23.702646,Snap Shot
237729,Goal,1,671.0,671,81.0,-7.0,10.630146,-41.185925,Wrist Shot
16368,Goal,1,654.0,654,84.0,-9.0,10.295630,-60.945396,Tip-In
245214,Shot,2,725.0,1925,-52.0,19.0,142.274383,7.674480,Slap Shot
171771,Shot,2,169.0,1369,80.0,8.0,12.041595,41.633539,Backhand
278405,Shot,4,235.0,3835,46.0,-10.0,44.147480,-13.091893,Snap Shot
151992,Shot,1,510.0,510,23.0,33.0,73.790243,26.565051,Wrist Shot
234627,Shot,1,1162.0,1162,29.0,18.0,62.641839,16.699244,Slap Shot


In [42]:
# Q2 
features_data_df[['Type', 'Game seconds', 'Last event type', 'Last event elapsed time', 'Last event distance', 'Last event st_X', 'Last event st_Y']].sample(10)

,Type,Game seconds,Last event type,Last event elapsed time,Last event distance,Last event st_X,Last event st_Y
271165,Shot,2709,Hit,23.5,42.154478,97.0,-21.0
103165,Shot,2363,Faceoff,6.5,45.453273,69.0,-22.0
278379,Shot,2109,Faceoff,15.5,41.340053,69.0,-22.0
163796,Shot,2354,Shot,0.5,22.022716,85.0,23.0
153968,Shot,2888,Faceoff,28.5,61.294372,-20.0,-22.0
9608,Shot,3044,Hit,43.5,35.846897,99.0,13.0
91458,Shot,3520,Faceoff,13.5,19.849433,69.0,-22.0
137051,Goal,562,Missed Shot,0.5,30.000000,55.0,-14.0
261846,Shot,50,Shot,12.5,111.004504,-54.0,-33.0
124904,Goal,3538,Missed Shot,9.5,35.014283,-40.0,-3.0


In [43]:
# Q3
features_data_df[['Type', 'Last event type', 'Last event elapsed time', 'Last event distance', 'Last event st_X', 'Last event st_Y', 'Rebound', 'Change in Shot Angle', 'Speed From Previous Event']].sample(10)

,Type,Last event type,Last event elapsed time,Last event distance,Last event st_X,Last event st_Y,Rebound,Change in Shot Angle,Speed From Previous Event
51551,Shot,Hit,26.5,35.510562,97.0,-12.0,0.0,0.000000,1.340021
49680,Shot,Faceoff,4.5,22.360680,69.0,22.0,0.0,0.000000,4.969040
154948,Shot,Giveaway,5.5,106.705201,-26.0,12.0,0.0,0.000000,19.400946
13640,Shot,Shot,37.5,50.289164,70.0,12.0,1.0,78.912221,1.341044
39679,Shot,Hit,16.5,110.208893,-49.0,-40.0,0.0,0.000000,6.679327
2956,Shot,Missed Shot,11.5,34.713110,-65.0,5.0,0.0,0.000000,3.018531
246409,Shot,Faceoff,5.5,43.416587,20.0,22.0,0.0,0.000000,7.893925
85560,Shot,Hit,23.5,169.852878,-99.0,21.0,0.0,0.000000,7.227782
80294,Shot,Shot,3.5,34.828150,81.0,-5.0,1.0,68.259221,9.950900
45786,Shot,Faceoff,56.5,145.670862,-69.0,22.0,0.0,0.000000,2.578245


In [44]:
# Q4
features_data_df[['Type', 'Game seconds', 'Strength', 'Num players With', 'Num players Against', 'Elapsed time since Power Play']].query("Strength == 'Power Play'").sample(10)
# features_data_df.columns

,Type,Game seconds,Strength,Num players With,Num players Against,Elapsed time since Power Play
173085,Goal,1237,Power Play,5.0,4.0,16.0
213422,Shot,2869,Power Play,5.0,4.0,66.0
184263,Shot,486,Power Play,5.0,4.0,8.0
18443,Shot,1654,Power Play,5.0,4.0,49.0
151602,Shot,1757,Power Play,5.0,4.0,42.0
255566,Shot,2460,Power Play,5.0,4.0,93.0
96269,Shot,2008,Power Play,4.0,3.0,235.0
175691,Shot,763,Power Play,5.0,4.0,119.0
74304,Shot,346,Power Play,5.0,4.0,127.0
180489,Goal,1691,Power Play,5.0,4.0,102.0
